# sk_learn Outlier removal

In [1]:
import pandas as pd;
import numpy as np;
from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelBinarizer, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pd.set_option('display.float_format', lambda x: '%.1f' % x)

In [2]:
### CUSTOM OUTLIER REMOVAL TRANSFORMERS ###
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.exceptions import NotFittedError
from typing import Tuple, Callable, List

# Decorator for outlier removers: 
# src: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html
outlier_filter \
    : Callable[[str, Callable, List[str]|None], Tuple[str, FunctionTransformer]] \
    = lambda desc, strategy_fn, columns=None: (desc, FunctionTransformer(outlier_remover(strategy_fn, columns)))

# Will receive a filter-strategy and remove all rows that don't match the filter
def outlier_remover(apply_filter_to_feature: Callable[[pd.DataFrame, str], bool], columns=None):     
    def filter(D: pd.DataFrame): 
        filters = np.all([
            apply_filter_to_feature(D, f)  # Remove where value is not
            for f in columns or D.columns
        ], axis=0)
        return D[filters].copy()
    return filter

## AVAILABLE CONTINUES FILTER STRATEGIES ##
def iqr_strategy(factor=1.5):
    def fit( D: pd.DataFrame, f: str):
        Q1 = D[f].quantile(0.25)
        Q3 = D[f].quantile(0.75)
        bound = factor * (Q3 - Q1) # factor * IQR
        return (D[f] >= Q1 - bound) & (D[f] <= Q3 + bound)
    return fit

def z_score_strategy(factor=3):
    def fit( D: pd.DataFrame, f: str):
        mean = D[f].mean()
        bound = factor * D[f].std()
        return (D[f] >= mean - bound) & (D[f] <= mean + bound)
    return fit

def minmax_strategy(min, max):
    return lambda D, f: (D[f] >= min) & (D[f] <= max)

## AVAILABLE CATEGORICAL FILTER STRATEGIES ##
def exclude_level_strategy(excluded_levels: List[str]):
    return lambda D, f: ~D[f].isin(excluded_levels)

def exclude_low_freq_strategy(threshold: float=0.2):
    def fit( D: pd.DataFrame, f: str):
        level_filter = D['job'].value_counts(normalize=True)>= threshold
        return D[f].isin(level_filter[level_filter].index)
    return fit

In [3]:
## CUSTOM TRANSFORMATOR TO MERGE AND GROUP VALUES ###
class TagFeatureLevels(BaseEstimator, TransformerMixin):
    def __init__(self, tags={}, columns=None):        
        self.tags = tags 
        self.columns = columns  
        self.fitted = False

    def _cols(self, X: pd.DataFrame): return self.columns or list(X.columns)

    def fit(self, X, y=None): 
        flatten_entry = lambda k, vals: {v: k for v in vals}
        flat_tags = reduce(lambda a, l: {**a, **flatten_entry(l[0], l[1])}, self.tags.items(), {})
        self._replacemap = {c: flat_tags for c in self._cols(X)}
        self.fitted = True
        return self

    def transform(self,X: pd.DataFrame,y=None):
        if not self.fitted: raise NotFittedError()
        return X.replace(self._replacemap)

In [4]:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html
multicolumn_encoder = lambda step_desc, encoder, columns=None: (step_desc, FunctionTransformer(encoding_mapper(encoder.fit_transform, columns),encoding_mapper(encoder.inverse_transform, columns)))

def encoding_mapper(encoder, columns=None): 
    def fit_transform(X): 
        encoded_X = X.copy()
        cols = columns or X.columns
        encoded_X[cols] = [encoder(encoded_X) for c in cols]
        return encoded_X
    
    return fit_transform

## Using the pipeline to clean datasets

In [5]:
headers = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'ft_income']
df = pd.read_csv('../../datasets/adults.csv', engine='python', sep=', ', names=headers, na_values=['?'])

# df.head(5)
df[["workclass", "occupation", "native-country"]].isna().sum()

workclass         1836
occupation        1843
native-country     583
dtype: int64

In [6]:
X = df.drop(['ft_income'],axis=1)
y = df['ft_income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42 ) 
steps = []
X_train

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
5514,33,Local-gov,198183,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,50,United-States
19777,36,Private,86459,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,50,United-States
10781,58,Self-emp-not-inc,203039,9th,5,Separated,Craft-repair,Not-in-family,White,Male,0,0,40,United-States
32240,21,Private,180190,Assoc-voc,11,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,46,United-States
9876,27,Private,279872,Some-college,10,Divorced,Other-service,Not-in-family,White,Male,0,0,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29802,47,Private,359461,Bachelors,13,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States
5390,31,Private,147215,12th,8,Divorced,Other-service,Unmarried,White,Female,0,0,21,United-States
860,18,Private,216284,11th,7,Never-married,Adm-clerical,Own-child,White,Female,0,0,20,United-States
15795,50,Self-emp-not-inc,54261,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,84,United-States


In [7]:
steps.extend([
    outlier_filter('IQR outlier removal',       iqr_strategy(factor=1.5),           columns=["fnlwgt", "hours-per-week"]),
    outlier_filter('Threshold outlier removal', minmax_strategy(min=0, max=30000),  columns=["capital-gain"]),
])

### Transform columns

In [8]:
tf_impute = ["occupation", "native-country"]
tf_encode = ["sex"]
tf_merge = ["education"]
tf_scale = ['fnlwgt','capital-gain', 'capital-loss', 'hours-per-week']
tf_pass = ['age', 'workclass', 'marital-status','relationship', 'race']

transformers = [
    #Categorical
    ('Impute missing workclass', SimpleImputer(strategy='most_frequent'), tf_impute),
    ('encode binary columns', OrdinalEncoder(categories=[['Male', 'Female']]), tf_encode),
    ('Merge education levels',
        TagFeatureLevels(tags={ 
            "MS-dropout": ['7th-8th','5th-6th','1st-4th'], 
            "HS-dropout": ['9th','10th','11th','12th']
        }),
        tf_merge
    ),
    # Continues
    ('Continues features', StandardScaler(), tf_scale),
    # Skip others  
    ('Keep clean columns','passthrough',tf_pass)
]

order = tf_impute + tf_encode + tf_merge + tf_scale + tf_pass

In [9]:
# transformer = TagFeatureLevels(
#     tags={ 
#         "MS-dropout": ['7th-8th','5th-6th','1st-4th'], 
#         "HS-dropout": ['9th','10th','11th','12th']
#     }, 
#     columns=["education"]
# )
# X_train = transformer.fit_transform(X_train)

# print(f"before: {X_train.shape[0]} rows")
# transformerA = OutlierRemover(strat=iqr_score_strategy(factor=1.5), columns=["fnlwgt", "hours-per-week"])
# X_train = transformerA.fit_transform(X_train)
# transformerB = OutlierRemover(strat=minmax_strategy(min=0, max=30_000), columns=["capital-gain"])
# X_train = transformerB.fit_transform(X_train)
# print(f"after: {X_train.shape[0]} rows")

# imputed = pd.DataFrame(X_train, columns=X_train.columns)
# imputed["education"].unique()

In [10]:
steps.append((
    'Column transformer',
    ColumnTransformer( 
        transformers=transformers, 
        remainder='drop', # education-num
        verbose=True
    )
))

pipeline = Pipeline(steps=steps, verbose=True)

print(f"before: {X_train.shape[0]} rows")
clean_data = pipeline.fit_transform(X_train)
print(f"after: {clean_data.shape[0]} rows")

X_clean = pd.DataFrame(
    clean_data, 
    columns=order
)


before: 26048 rows
[Pipeline]  (step 1 of 3) Processing IQR outlier removal, total=   0.0s
[Pipeline]  (step 2 of 3) Processing Threshold outlier removal, total=   0.0s
[ColumnTransformer]  (1 of 5) Processing Impute missing workclass, total=   0.0s
[ColumnTransformer]  (2 of 5) Processing encode binary columns, total=   0.0s
[ColumnTransformer]  (3 of 5) Processing Merge education levels, total=   0.0s
[ColumnTransformer]  (4 of 5) Processing Continues features, total=   0.0s
[ColumnTransformer]  (5 of 5) Processing Keep clean columns, total=   0.0s
[Pipeline]  (step 3 of 3) Processing Column transformer, total=   0.0s
after: 18199 rows


In [11]:
def categorial(dataset, features):
    headers = ["feature","count", "% miss.", "card.", "Mode", "Mode freq.", "Mode %", "2nd Mode", "2nd Mode freq.", "2nd Mode %"]
    report = [ [
        f,
        dataset[f].size,
        dataset[f].isnull().sum() / dataset[f].size,
        dataset[f].nunique(),
        dataset[f].mode().values[0],
        dataset[f].value_counts().max(),
        (dataset[f].value_counts().max() / dataset[f].size) * 100,
        
        dataset[f].value_counts().index[1],
        dataset[f].value_counts().iloc[1],
        (dataset[f].value_counts().iloc[1] / dataset[f].size) * 100,
    ] for f in features]
    
    return pd.DataFrame(report, columns=headers)

categorial_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

In [12]:
X_train.pipe(categorial, categorial_cols)

,feature,count,% miss.,card.,Mode,Mode freq.,Mode %,2nd Mode,2nd Mode freq.,2nd Mode %
0,workclass,26048,0.1,8,Private,18118,69.6,Self-emp-not-inc,2043,7.8
1,education,26048,0.0,16,HS-grad,8416,32.3,Some-college,5806,22.3
2,marital-status,26048,0.0,7,Married-civ-spouse,12026,46.2,Never-married,8557,32.9
3,occupation,26048,0.1,14,Prof-specialty,3312,12.7,Craft-repair,3278,12.6
4,relationship,26048,0.0,6,Husband,10603,40.7,Not-in-family,6603,25.3
5,race,26048,0.0,5,White,22221,85.3,Black,2525,9.7
6,sex,26048,0.0,2,Male,17403,66.8,Female,8645,33.2
7,native-country,26048,0.0,41,United-States,23300,89.5,Mexico,529,2.0


In [13]:
X_clean.pipe(categorial, categorial_cols)

,feature,count,% miss.,card.,Mode,Mode freq.,Mode %,2nd Mode,2nd Mode freq.,2nd Mode %
0,workclass,18199,0.0,8,Private,13100,72.0,Local-gov,1311,7.2
1,education,18199,0.0,11,HS-grad,6231,34.2,Some-college,3815,21.0
2,marital-status,18199,0.0,7,Married-civ-spouse,8842,48.6,Never-married,5392,29.6
3,occupation,18199,0.0,14,Craft-repair,3399,18.7,Exec-managerial,2361,13.0
4,relationship,18199,0.0,6,Husband,7856,43.2,Not-in-family,4719,25.9
5,race,18199,0.0,5,White,15446,84.9,Black,1804,9.9
6,sex,18199,0.0,2,0.0,12455,68.4,1.0,5744,31.6
7,native-country,18199,0.0,41,United-States,16570,91.0,Mexico,360,2.0


In [14]:
continues_features = ['fnlwgt','age','capital-gain', 'capital-loss', 'hours-per-week']
X_train[continues_features].describe()

,fnlwgt,age,capital-gain,capital-loss,hours-per-week
count,26048.0,26048.0,26048.0,26048.0,26048.0
mean,189759.2,38.6,1053.6,88.0,40.4
std,105232.9,13.6,7230.8,403.6,12.3
min,12285.0,17.0,0.0,0.0,1.0
25%,117963.0,28.0,0.0,0.0,40.0
50%,178263.5,37.0,0.0,0.0,40.0
75%,237006.5,48.0,0.0,0.0,45.0
max,1484705.0,90.0,99999.0,4356.0,99.0


In [15]:
X_clean[continues_features] = X_clean[continues_features].astype(float)
X_clean[continues_features].describe()

,fnlwgt,age,capital-gain,capital-loss,hours-per-week
count,18199.0,18199.0,18199.0,18199.0,18199.0
mean,-0.0,39.0,-0.0,0.0,0.0
std,1.0,12.5,1.0,1.0,1.0
min,-1.9,17.0,-0.2,-0.2,-2.1
25%,-0.7,29.0,-0.2,-0.2,-0.4
50%,-0.0,38.0,-0.2,-0.2,-0.4
75%,0.6,47.0,-0.2,-0.2,0.1
max,2.7,90.0,10.9,10.5,2.6


In [16]:
print(X_train["education"].unique())
print(X_clean["education"].unique())

['Bachelors' 'Assoc-voc' '9th' 'Some-college' '10th' 'HS-grad'
 'Prof-school' 'Assoc-acdm' '11th' '12th' 'Masters' '7th-8th' 'Doctorate'
 '5th-6th' '1st-4th' 'Preschool']
['Bachelors' 'Assoc-voc' 'HS-dropout' 'Some-college' 'HS-grad'
 'Assoc-acdm' 'MS-dropout' 'Doctorate' 'Prof-school' 'Masters' 'Preschool']


In [17]:
print(X_train["workclass"].unique())
print(X_clean["workclass"].unique())

['Local-gov' 'Private' 'Self-emp-not-inc' nan 'Federal-gov' 'Self-emp-inc'
 'State-gov' 'Without-pay' 'Never-worked']
['Local-gov' 'Private' 'Self-emp-not-inc' 'Federal-gov' nan 'Self-emp-inc'
 'State-gov' 'Without-pay' 'Never-worked']
